In [80]:
ORM_TYPE="postgres"
ORM_HOST="163.152.71.164"
ORM_PORT="51432"
ORM_USERNAME="user"
ORM_PASSWORD="xhahr123!"
ORM_DATABASE="postgres"
ORM_TABLENAME="nimbus"

In [81]:
from urllib.parse import urlparse
from langchain.vectorstores.pgvector import PGVector
import psycopg2
from tomok import RuleUnitController
import json
import re
import os

def connect_simple():
    url = PGVector.connection_string_from_db_params(
        driver='psycopg2',
        host=ORM_HOST,
        port=ORM_PORT,
        database=ORM_DATABASE,
        user=ORM_USERNAME,
        password=ORM_PASSWORD
    )
    result = urlparse(url)
    conn = psycopg2.connect(
        database=result.path[1:],
        user=result.username,
        password=result.password,
        host=result.hostname,
        port=result.port
    )
    return conn

In [82]:
def find_py_files_and_content(root_dir):
    files_and_contents = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.py'):
                # 파일 경로에서 root 디렉토리를 제거
                relative_path = os.path.relpath(os.path.join(root, file), root_dir)
                # 파일 이름에서 .py 확장자 제거
                base_name = os.path.splitext(file)[0]
                # 경로를 디렉토리와 파일 이름으로 분할
                directories = relative_path.split(os.sep)[:-1]  # 파일 이름 제외
                directories.append(base_name)

                # 파일 내용 읽기
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                # 경로와 내용을 튜플로 묶어 리스트에 추가
                files_and_contents.append((directories, content))

    return files_and_contents
# 사용 예
root_directory = '../ruleunits'  # 여기에 원하는 디렉토리 경로를 넣으세요.
py_paths_contents  = find_py_files_and_content(root_directory)[1:]


def build_classname(dir_list):
    return dir_list[0].upper()+dir_list[1]+'_'+dir_list[2]

# # 출력 예
# for path, contents in py_paths_contents[:3]:
#     if len(path) > 2:
#         print(path)
#         print(build_classname(path))
#         print(contents)

In [83]:
def get_values(ru):
    fc = ru.flowchart
    metadata = {}
    metadata['priority'] = ru.priority
    metadata['version'] = ru.version
    metadata['author'] = ru.author
    metadata['ref_code'] = ru.ref_code
    metadata['ref_date'] = ru.ref_date
    metadata['doc_date'] = ru.doc_date
    metadata['title'] = ru.title
    metadata['description'] = ru.description
    metadata['content'] = ru.content
    content = ru.content
    
    return fc, metadata, content

In [84]:
controller = RuleUnitController(path='../ruleunits')

values = []
contents = []

for path, content in py_paths_contents:
    if len(path) > 2:
        try:
            values.append(get_values(
                controller[path[0]][path[1]][path[2]][build_classname(path)]
            ))
            contents.append(content)
        except:
            continue

In [85]:
def get_function_info(content: str):
    lines = content.split('\n')
    
    f_string = ""
    flag = False
    
    for line in lines[:-3]:
        if flag:
            f_string += "\n" + line
    
        if "@rule_method" in line:
            flag = True
    
    f_string = f_string.strip()
    # print(f_string)

    lines = f_string.split('\n')
    function_def = lines[0].split(' ')[1].split('(')
    function_name = function_def[0]
    function_vars = function_def[1][:-1]
    
    return f_string, function_name, function_vars

get_function_info(contents[0])

('def Strength_of_longitudinally_reinforcedstruts(fOFns,fIfprims,fIAprims,fIAc,fIfce) -> float:\n        """종방향으로 보강된 스트럿의 강도\n\n        Args:\n            fOFns (float): 종방향으로 보강된 스트럿의 강도\n            fIfprims (float): 압축철근의 응력\n            fIAprims (float): 철근스트럿의 단면적\n            fIAc (float): 스트럿의 유효단면적\n            fIfce (float): 스트럿 또는 절점영역의 콘크리트 유효압축강도\n\n        Returns:\n            float: 콘크리트 스트럿-타이모델 기준  4.2.4 철근 효과 (2)의 값\n        """\n        fOFns=fIfce*fIAc+1.13*fIAprims*fIfprims\n        return fOFns',
 'Strength_of_longitudinally_reinforcedstruts',
 'fOFns,fIfprims,fIAprims,fIAc,fIfce')

In [86]:
values[0][1]['ref_code'][:12]

'KDS 14 20 24'

In [87]:
values[30][1]['description'].split("\n")[1].strip()

'강교 설계기준(한계상태설계법)'

In [88]:
name = values[30][1]['description'].strip().split("\n")[-2].strip().split(' ', 1)[1]

name_lines = values[30][1]['description'].strip().split("\n")

for x in range(len(name_lines)-1, -1, -1):
    if name_lines[x].strip()[0] != "(":
        name = name_lines[x].strip().split(' ', 1)[1]
        break
    
print(name)

바닥판과 바닥틀을 설계하는 경우의 설계차량활하중


In [89]:
for i in range(len(values)):
    try:
        conn = connect_simple()
        cur = conn.cursor()
        
        code = values[i][1]['ref_code'][:12]
        cur.execute("SELECT std_book_id FROM standard_book WHERE code = %s", (code,))
        book_id = cur.fetchone()[0]  
        print(code, book_id)
            
        insert_query = """INSERT INTO standard_item (std_id, std_book_id, name, code, digital, content, item_order) VALUES 
        (uuid_generate_v4(), %s, %s, %s, %s, %s, %s)
        """
        
        name_lines = values[i][1]['description'].strip().split("\n")
        for x in range(len(name_lines)-1, -1, -1):
            if name_lines[x].strip()[0] != "(":
                name = name_lines[x].strip().split(' ', 1)[1]
                break
            
        print(name)
        code = values[i][1]['ref_code']
        content = values[i][1]['content']
        
        cur.execute(insert_query, (book_id, name, code, True, content, 0))
        conn.commit()
    except psycopg2.Error as e:
        # Handle psycopg2-related errors
        print(f"SQL error on index {i}: {e.pgcode} - {e.pgerror}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()

KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
철근 효과
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
유효압축강도
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
유효압축강도
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
축강도 산정
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
축강도 산정
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
강도 산정
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
유효압축강도
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
설계 원칙
KDS 14 20 24 be939a80-31ff-47bf-a8ed-60c288690be5
유효압축강도
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
파괴확률
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
원심하중
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
장대레일 종하중 : LR
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
바지선의 이물 손상길이
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
종방향
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
연간파괴빈도
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
항로이탈확률
KDS 24 12 21 07e89952-2e05-41b2-bdf3-0478ee94203f
선박충돌에너지
KDS 24 12 21 07e89952-2e05-

In [90]:
values[0][1]

{'priority': 1,
 'version': 1.0,
 'author': 'Jiwoo Won',
 'ref_code': 'KDS 14 20 24 4.2.4 (2)',
 'ref_date': '2021-02-18',
 'doc_date': '2023-11-21',
 'title': '종방향으로 보강된 스트럿의 강도',
 'description': '\n    콘크리트구조 스트럿-타이모델 기준\n    4. 설계\n    4.2 스트럿의 축강도\n    4.2.4 철근 효과\n    (2)\n    ',
 'content': '\n    '}